reasons why you need to be using CoPilot. (and know how to google)

we need to build a python function that can query APIs and convert it into a dataset which we can then apply NLP or whatever Grant SMith wants...

first step... model the JSON in a series of classes.

annnnnd done.


In [61]:
# exporti
from enum import Enum, auto
from nbdev.showdoc import patch_to
from typing import List
from datetime import datetime
from typing import List
from dataclasses import dataclass, field
from typing import Optional, Dict

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [62]:
sample_discussion = {
    "discussionID": 62887,
    "type": "question",
    "name": "How do I set up multiple case statement for required field?",
    "body": '<p>I\'m trying to create a case statement to replace an empty field with the word \'incomplete\' when the field in the previous \'required\' column has a \'yes\' response. When it has a \'no\' response, it should be left blank. Here is a snip of the dataset.</p><div class="embedExternal embedImage display-large float-none" data-embedjson="{&quot;url&quot;:&quot;https:\/\/us.v-cdn.net\/6032830\/uploads\/ITXWLKPOYUYV\/image.png&quot;,&quot;name&quot;:&quot;image.png&quot;,&quot;type&quot;:&quot;image\/png&quot;,&quot;size&quot;:18937,&quot;width&quot;:414,&quot;height&quot;:480,&quot;displaySize&quot;:&quot;large&quot;,&quot;float&quot;:&quot;none&quot;,&quot;embedType&quot;:&quot;image&quot;,&quot;embedStyle&quot;:&quot;rich_embed_card&quot;}">\n    <div class="embedExternal-content">\n        <a class="embedImage-link" href="https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png" rel="nofollow noopener ugc" target="_blank">\n            <img class="embedImage-img" src="https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png" alt="image.png" height="480" width="414" loading="lazy" data-display-size="large" data-float="none" data-type="image/png" data-embed-type="image" srcset="https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=10/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png 10w, https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=300/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png 300w, https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=800/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png 800w, https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=1200/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png 1200w, https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png"></img></a>\n    </div>\n</div>\n<p>This is what I\'ve got so far, but I realize the \'if null\' statement doesn\'t work this way.</p><p>CASE<br>&#13;\nWHEN<br><code class="code codeInline" spellcheck="false" tabindex="0">Oct \'23 Required</code>= \'yes\'<br>&#13;\nAND<br>&#13;\nIFNULL(<code class="code codeInline" spellcheck="false" tabindex="0">Oct \'23</code>)<br>&#13;\nTHEN \'incomplete\'<br>&#13;\nELSE <code class="code codeInline" spellcheck="false" tabindex="0">Oct \'23</code><br>&#13;\nEND</p>',
    "categoryID": 181,
    "dateInserted": "2023-11-11T00:03:25+00:00",
    "dateUpdated": None,
    "dateLastComment": "2023-11-11T00:39:03+00:00",
    "insertUserID": 40460,
    "insertUser": {
        "userID": 40460,
        "name": "Justice_RUCD",
        "url": "https://community-forums.domo.com/profile/Justice_RUCD",
        "photoUrl": "https://us.v-cdn.net/6032830/uploads/defaultavatar/n5PIVLOQ8FU41.jpg",
        "dateLastActive": "2023-11-11T00:03:26+00:00",
        "banned": 0,
        "punished": 0,
        "private": False,
        "label": "Member",
    },
    "updateUserID": None,
    "lastUserID": 38834,
    "lastUser": {
        "userID": 38834,
        "name": "ColemenWilson",
        "url": "https://community-forums.domo.com/profile/ColemenWilson",
        "photoUrl": "https://us.v-cdn.net/6032830/uploads/userpics/30IQ3R6A617R/nYJEIZXVA579B.png",
        "dateLastActive": "2023-11-11T17:18:32+00:00",
        "banned": 0,
        "punished": 0,
        "private": False,
        "label": 'Coach <img src="https://us.v-cdn.net/6033814/uploads/M39SL97E7E4L/rank-coach.png">',
    },
    "pinned": False,
    "pinLocation": None,
    "closed": False,
    "sink": False,
    "countComments": 1,
    "countViews": 17,
    "score": None,
    "hot": 3399324748,
    "url": "https://community-forums.domo.com/main/discussion/62887/how-do-i-set-up-multiple-case-statement-for-required-field",
    "canonicalUrl": "https://community-forums.domo.com/main/discussion/62887/how-do-i-set-up-multiple-case-statement-for-required-field",
    "format": "Rich2",
    "groupID": None,
    "statusID": 2,
    "image": {
        "url": "https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png",
        "urlSrcSet": {
            "10": "https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=10/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png",
            "300": "https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=300/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png",
            "800": "https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=800/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png",
            "1200": "https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=1200/https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png",
            "1600": "https://us.v-cdn.net/6032830/uploads/ITXWLKPOYUYV/image.png",
        },
        "alt": "image.png",
    },
    "attributes": {
        "question": {
            "status": "answered",
            "dateAccepted": None,
            "dateAnswered": "2023-11-11T00:39:03+00:00",
            "acceptedAnswers": [],
        }
    },
    "bookmarked": False,
    "unread": False,
}

In [63]:
sample_category = {'categoryID': 145,
 'name': 'Community Forums',
 'description': 'Discover best practices and find answers to your questions with advice from fellow Domo users.',
 'parentCategoryID': None,
 'customPermissions': False,
 'isArchived': False,
 'urlcode': 'main',
 'url': 'https://community-forums.domo.com/main/categories',
 'displayAs': 'categories',
 'iconUrl': None,
 'dateInserted': '2022-05-27T16:03:36+00:00',
 'bannerUrl': 'https://us.v-cdn.net/6032830/uploads/OA4A2CR9RRVI/banner-blue-overlay.png',
 'bannerUrlSrcSet': {'10': 'https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=10/https://us.v-cdn.net/6032830/uploads/OA4A2CR9RRVI/banner-blue-overlay.png',
  '300': 'https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=300/https://us.v-cdn.net/6032830/uploads/OA4A2CR9RRVI/banner-blue-overlay.png',
  '800': 'https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=800/https://us.v-cdn.net/6032830/uploads/OA4A2CR9RRVI/banner-blue-overlay.png',
  '1200': 'https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=1200/https://us.v-cdn.net/6032830/uploads/OA4A2CR9RRVI/banner-blue-overlay.png',
  '1600': 'https://us.v-cdn.net/cdn-cgi/image/fit=scale-down,width=1600/https://us.v-cdn.net/6032830/uploads/OA4A2CR9RRVI/banner-blue-overlay.png'},
 'countCategories': 16,
 'countDiscussions': 0,
 'countComments': 0,
 'countAllDiscussions': 27047,
 'countAllComments': 75961,
 'countFollowers': 0,
 'followed': False,
 'breadcrumbs': [{'name': 'Home', 'url': 'https://community-forums.domo.com/'},
  {'name': 'Community Forums',
   'url': 'https://community-forums.domo.com/main/categories'}],
 'featured': False,
 'allowedDiscussionTypes': ['discussion', 'question'],
 'depth': 1,
 'sort': 2}

# Utility Functions

In [64]:
#|exporti
def sort_dictionary(my_dict):
    my_keys = list(my_dict.keys())
    my_keys.sort()
    return {key: my_dict[key] for key in my_keys}
 

def convert_string_to_datetime(date_string: Optional[str]) -> Optional[datetime]:
    """
    Convert a string date to a datetime object.

    Args:
        date_string (Optional[str]): The string date.

    Returns:
        Optional[datetime]: The converted datetime object, or None if date_string is None.
    """
    return datetime.fromisoformat(date_string) if date_string else None

def convert_datetime_to_string(date_date, format_str = "%Y-%m-%d %H:%M"):
    return date_date.strftime(format_str) if date_date else None

def clean_html_string(html_string: str) -> str:
    """
    Clean an HTML string using BeautifulSoup.

    Args:
        html_string (str): The HTML string.

    Returns:
        str: The cleaned string.
    """
    soup = BeautifulSoup(html_string, "html.parser")
    clean_string = soup.get_text()
    return clean_string

# Class Representation

In [65]:
@dataclass
class Breadcrumb:
    name: str
    url: str

    @classmethod
    def _from_json(cls, json_obj):
        return cls(name = json_obj['name'], url = json_obj['url'])

@dataclass
class UrlSrcSet:
    urls: Dict[str, str]

    @classmethod
    def _from_json(cls, data):
        return cls(urls=data)

@dataclass
class Category:
    base_url: str
    category_id: int
    name: str
    description: str
    parent_category_id: int
    # custom_permissions: bool
    # is_archived: bool

    # urlcode: str
    url: str
    display_as: str
    # icon_url: str

    date_inserted: datetime
    # banner_url: str
    # banner_url_src_set: UrlSrcSet

    count_categories: int
    count_discussions: int
    count_comments: int
    count_all_discussions: int
    count_all_comments: int
    count_followers: int

    # followed: bool


    # featured: bool
    allowed_discussion_types: List[str]

    sort: int = None
    depth: int = None

    # icon_url_src_set: UrlSrcSet
    breadcrumbs: List[Breadcrumb] = None

    @classmethod
    def _from_json(cls, json_obj, base_url):
        category =  cls(
            base_url = base_url,
            category_id=json_obj['categoryID'],
            name=json_obj['name'],
            description=json_obj['description'],
            parent_category_id=json_obj['parentCategoryID'],
            url=json_obj['url'],
            display_as=json_obj['displayAs'],
            
            date_inserted=convert_string_to_datetime(json_obj['dateInserted']),

            # urlcode=json_obj['urlcode'],
            # banner_url=json_obj['bannerUrl'],
            # banner_url_src_set=UrlSrcSet._from_json(json_obj['bannerUrlSrcSet']),
            
            count_categories=json_obj['countCategories'],
            count_discussions=json_obj['countDiscussions'],
            count_comments=json_obj['countComments'],
            count_all_discussions=json_obj['countAllDiscussions'],
            count_all_comments=json_obj['countAllComments'],
            count_followers=json_obj['countFollowers'],
            allowed_discussion_types=json_obj['allowedDiscussionTypes'],
            depth=json_obj.get('depth'),
            sort=json_obj.get('sort')
        )

        category.breadcrumbs = [ Breadcrumb._from_json(breadcrumb) for breadcrumb in json_obj['breadcrumbs']]
        return category

    @classmethod
    def get_by_id(cls, base_url, category_id, return_raw: bool = False):
        url = f"https://{base_url}/api/v2/categories/{category_id}"

        res = requests.get(url)

        if res.status_code != 200:
            raise Exception(f'unable to retrieve category {category_id}')
        
        if return_raw:
            return res

        json_obj = res.json()
        return Category._from_json(json_obj ,base_url= base_url)
        

In [66]:
Category.get_by_id('community-forums.domo.com', 145)

Category(base_url='community-forums.domo.com', category_id=145, name='Community Forums', description='Discover best practices and find answers to your questions with advice from fellow Domo users.', parent_category_id=None, url='https://community-forums.domo.com/main/categories', display_as='categories', date_inserted=datetime.datetime(2022, 5, 27, 16, 3, 36, tzinfo=datetime.timezone.utc), count_categories=16, count_discussions=0, count_comments=0, count_all_discussions=39759, count_all_comments=105948, count_followers=0, allowed_discussion_types=['discussion', 'question'], sort=None, depth=None, breadcrumbs=[Breadcrumb(name='Home', url='https://community-forums.domo.com/'), Breadcrumb(name='Community Forums', url='https://community-forums.domo.com/main/categories')])

In [67]:
#|export

@dataclass
class User:
    user_id: int
    name: str
    url: str
    photo_url: str
    date_last_active: str
    banned: int
    punished: int
    private: bool
    label: str

    @classmethod
    def _from_json(cls, json_obj):
        return cls(
            user_id=json_obj["userID"],
            name=json_obj["name"],
            url=json_obj["url"],
            photo_url=json_obj["photoUrl"],
            date_last_active=convert_string_to_datetime(json_obj["dateLastActive"]),
            banned=json_obj["banned"],
            punished=json_obj["punished"],
            private=json_obj["private"],
            label=json_obj["label"],
        )

In [68]:
User._from_json(sample_discussion["lastUser"])

User(user_id=38834, name='ColemenWilson', url='https://community-forums.domo.com/profile/ColemenWilson', photo_url='https://us.v-cdn.net/6032830/uploads/userpics/30IQ3R6A617R/nYJEIZXVA579B.png', date_last_active=datetime.datetime(2023, 11, 11, 17, 18, 32, tzinfo=datetime.timezone.utc), banned=0, punished=0, private=False, label='Coach <img src="https://us.v-cdn.net/6033814/uploads/M39SL97E7E4L/rank-coach.png">')

In [69]:
#| export

@dataclass
class Image:
    url: str
    urlSrcSet: Dict[str, str]
    alt: str

In [70]:
# | export


class QuestionStatusEnum(Enum):
    ANSWERED = auto()
    UNANSWERED = auto()


@dataclass
class Attribute:
    pass


@dataclass
class Question(Attribute):
    status: QuestionStatusEnum = None
    date_accepted: Optional[datetime] = None
    date_answered: datetime = None
    accepted_answers: List[str] = None

    @classmethod
    def _from_json(cls, json_obj):
        """
        Create a Question object from a JSON object.

        Args:
            json_obj (dict): The JSON object.

        Returns:
            Question: The created Question object.
        """
        status = (
            QuestionStatusEnum[json_obj["status"].upper()]
            if json_obj["status"].upper() in QuestionStatusEnum.__members__
            else None
        )
        date_accepted = convert_string_to_datetime(json_obj["dateAccepted"])
            
        date_answered = convert_string_to_datetime(json_obj["dateAnswered"])
        return cls(
            status=status,
            date_accepted=date_accepted,
            date_answered=date_answered,
            accepted_answers=json_obj["acceptedAnswers"],
        )

In [71]:
Question._from_json(sample_discussion["attributes"]["question"])

Question(status=<QuestionStatusEnum.ANSWERED: 1>, date_accepted=None, date_answered=datetime.datetime(2023, 11, 11, 0, 39, 3, tzinfo=datetime.timezone.utc), accepted_answers=[])

In [ ]:
# | export


@dataclass
class Discussion:
    base_url: str
    discussion_id: int
    type: str
    name: str
    body: str
    category_id: int
    date_inserted: datetime
    date_updated: Optional[datetime]
    date_last_comment: str
    insert_user_id: int
    insert_user: User
    closed: bool
    url: str
    canonical_url: str
    status_id: int
    score: Optional[int]

    # count_comments: int
    # count_views: int

    category: Optional[Category] = None
    body_clean: Optional[str] = None
    attributes: Optional[List[Question]] = field(default_factory=dict)

    def _attributes_from_json(self, attributes_dict: dict) -> Attribute:
        """
        Convert a list of attribute dictionaries to a list of Question objects.
        """
        if "question" in attributes_dict:
            self.attributes.update(
                {"question": Question._from_json(attributes_dict["question"])}
            )

        return self.attributes

    @classmethod
    def _from_json(cls, json_obj, base_url):
        discussion = cls(
            base_url=base_url,
            discussion_id=json_obj["discussionID"],
            type=json_obj["type"],
            name=json_obj["name"],
            body=json_obj["body"],
            category_id=json_obj["categoryID"],
            date_inserted=convert_string_to_datetime(json_obj.get("dateInserted")),
            date_updated=convert_string_to_datetime(json_obj.get("dateUpdated")),
            date_last_comment=convert_string_to_datetime(
                json_obj.get("dateLastComment")
            ),
            insert_user_id=json_obj["insertUserID"],
            insert_user=User._from_json(json_obj["insertUser"]),
            closed=json_obj["closed"],
            # count_comments=json_obj["countComments"],
            # count_views=json_obj["countViews"],
            score=json_obj["score"],
            url=json_obj["url"],
            canonical_url=json_obj["canonicalUrl"],
            status_id=json_obj["statusID"],
        )

        discussion.body_clean = clean_html_string(discussion.body)

        if "attributes" in json_obj:
            discussion._attributes_from_json(json_obj["attributes"])

        if "categoryID" in json_obj:
            discussion.category = Category.get_by_id(
                base_url=base_url, 
                category_id=json_obj["categoryID"]
            )

        return discussion

    def to_dict(self) -> dict:
        """
        Convert a Discussion object to a dictionary.

        Returns:
            dict: The dictionary representation of the Discussion object.
        """
        s= {
            "discussion_id": self.discussion_id,
            "discussion_type": self.type,
            "discussion_title": self.name,
            "discussion_body": self.body_clean,
            "category_id": self.category_id,
            "date_inserted": convert_datetime_to_string(self.date_inserted),
            "date_updated": convert_datetime_to_string(self.date_updated),
            "date_last_comment": convert_datetime_to_string(self.date_last_comment),
            "insert_user_id": self.insert_user_id,
            "closed": self.closed,
            # "count_comments": self.count_comments,
            # "count_views": self.count_views,
            "score": self.score,
            "url": self.url,
            "canonical_url": self.canonical_url,
            "status_id": self.status_id,
        }

        if self.category:
            s.update({'category_name' : self.category.name,
            'category_description' : self.category.description
             })
        
        if self.insert_user:
            s.update({'insert_user_name': self.insert_user.name, 'insert_user_profile' : self.insert_user.url
            })
    

        return sort_dictionary(s)

In [ ]:
discussion = Discussion._from_json(sample_discussion, base_url = 'community-forums.domo.com')

discussion.to_dict()

{'canonical_url': 'https://community-forums.domo.com/main/discussion/62887/how-do-i-set-up-multiple-case-statement-for-required-field',
 'category_description': 'Ask Beast Mode questions and get answers from Domo peers',
 'category_id': 181,
 'category_name': 'Beast Mode',
 'closed': False,
 'date_inserted': datetime.datetime(2023, 11, 11, 0, 3, 25, tzinfo=datetime.timezone.utc),
 'date_last_comment': datetime.datetime(2023, 11, 11, 0, 39, 3, tzinfo=datetime.timezone.utc),
 'date_updated': None,
 'discussion_body': "I'm trying to create a case statement to replace an empty field with the word 'incomplete' when the field in the previous 'required' column has a 'yes' response. When it has a 'no' response, it should be left blank. Here is a snip of the dataset.\n\n\n\n\n\nThis is what I've got so far, but I realize the 'if null' statement doesn't work this way.CASE\r\nWHENOct '23 Required= 'yes'\r\nAND\r\nIFNULL(Oct '23)\r\nTHEN 'incomplete'\r\nELSE Oct '23\r\nEND",
 'discussion_id': 6288

In [ ]:
# | export
@dataclass
class VanillaForums:
    base_url: str

    discussions: List[Discussion] = None

    def get_all_categories(self, page=1, return_raw: bool = False):
        url = f"https://{self.base_url}/api/v2/categories"
        params = {"page": page}

        res = requests.get(url, params=params)

        if res.status_code != 200:
            raise Exception('unable to retrieve categories')
        
        if return_raw:
            return res

        categories = res.json()
        self.categories = [Category._from_json(category, base_url = self.base_url) for category in categories]
        return self.categories


    def get_all_discussions(
        self,
        return_raw: bool = False,
        page: int = 1,
        is_only_unanswered: bool = False,
    ):

        params = {"sort": "-dateInserted", "page": page}

        if is_only_unanswered:
            params.update({"status": "unanswered"})

        url = f"https://{self.base_url}/api/v2/discussions"

        res = requests.get(url, params=params)

        if return_raw:
            return res

        if not res.status_code == 200:
            raise Exception("unable to retrieve discussions")

        discussions = res.json()

        self.discussions = [
            Discussion._from_json(discussion, base_url = self.base_url) for discussion in discussions
        ]
        return self.discussions

    def generate_discussion_report(self):
        if not self.discussions:
            self.get_all_discussions()
        
        return pd.DataFrame([ discussion.to_dict() for discussion in self.discussions])

In [ ]:
domo_community = VanillaForums(base_url = 'community-forums.domo.com')
domo_community.generate_discussion_report()


NameError: name 'NOne' is not defined

In [ ]:
domo_community = VanillaForums(base_url = 'community-forums.domo.com')

domo_community.generate_discussion_report()

AttributeError: 'Discussion' object has no attribute 'to_json'